In [1]:
#TO RE-RUN
%reset -f

In [2]:
from sklearn import preprocessing
from time import time
import numpy as np
import csv
from sklearn import metrics
from sklearn.preprocessing import scale
from sklearn.feature_selection import VarianceThreshold
from sklearn.cross_validation import StratifiedShuffleSplit, cross_val_score

from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB

from sklearn.grid_search import GridSearchCV, ParameterGrid
from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import SMOTE,ADASYN, RandomOverSampler
from imblearn.pipeline import Pipeline
from imblearn.pipeline import make_pipeline

from operator import truediv
from datetime import datetime
import pandas as pd
import time
import os

from pylab import *
import seaborn as sns
import matplotlib.pyplot as plt


np.set_printoptions(suppress=True)
pd.options.display.float_format = '{:,.2f}'.format
plt.style.use('classic')

%matplotlib inline

import sys
sys.path.insert(1, "../src/")
from TypeFeatImputer import TypeFeatImputer
from UnivCombineFilter import UnivCombineFilter
import MLpipeline as MLpipeline

/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


#### Run experiments

In [3]:
typeEncounter = "last" # ['first','last']
typeHypothesis = "all_readmisssion_vs_none" # ['all_readmisssion_vs_none','early_readmission_vs_none']
typeDataFeatures = "extended" # ["minimum","extended']
    #Extended -> Subset of columns
    #Minimum -> minimum set of columns 
typeDiagnosis = "none"  #["none","diag_1", "diag_3"]    
typeDisease = "subset" # ["subset","any",["Respiratory",...]]
    #subset -> Return subset of predefined disease features
    #any -> Return all disease features    
    #disease -> Return diseases feature
typeDataExperiment = "disease" #["all", "disease"] 
    #all -> Include all diagnosis as columns
    #disease -> Remove diagnosis as column and keep only rows with diagnosis == 1       

In [4]:
verbose = True
cv_thr = 0.3
cv_folds = 5

tr_thrs = [1.0] # [0.1,0.2,0.4,0.6,1.0]
ts_thr = 0.30

fs_methods = ["none"] #["none","combine_fs","lasso_fs","rfe_rf_fs"]
cls_methods = ["rf"] #["rf","svmRBF","logReg","knn","nn","gbt"]
lms = ["recall","f1_weighted"] #["f1_weighted","precision_weighted","roc_auc","recall]
sm_types = ["none"] #["none","after"]
sm_method = "sm_smote"

In [5]:
#Load data
df_all = MLpipeline.load_data(typeEncounter, typeDiagnosis, typeDataFeatures)
print "\nSHAPE:"
print df_all.shape
print "\nInitial columns:"
print df_all.columns

#Filter data by class
df_all = MLpipeline.filter_data_by_class(df_all, typeHypothesis)
print "\nRows by class type:"
print df_all.iloc[:,-1].sort_values().unique(), np.sum(df_all["readmitted"] == 0), np.sum(df_all["readmitted"] == 1)
    
#Get columns
colsDiseases, colsNonDiseases = MLpipeline.get_columns(df_all,typeDiagnosis)
print "\nDiseases:", colsDiseases
print "\nNon-diseases:", colsNonDiseases
    
#Load diseases
diseases = MLpipeline.get_diseases(colsDiseases, typeDisease)
print "\nTotal data:", df_all.shape
print diseases


SHAPE:
(67182, 52)

Initial columns:
Index([u'gender', u'age', u'race_AfricanAmerican', u'race_Caucasian',
       u'race_Other', u'HbA1c', u'Change', u'time_in_hospital',
       u'medSpec_cardio', u'medSpec_Family/GeneralPractice',
       u'medSpec_InternalMedicine', u'medSpec_surgery', u'number_treatment',
       u'number_treatment_0', u'number_treatment_1', u'number_treatment_2',
       u'number_treatment_3', u'adm_src_1', u'adm_src_2', u'adm_src_3',
       u'adm_src_4', u'adm_src_5', u'adm_src_6', u'adm_src_7', u'adm_src_8',
       u'adm_src_10', u'adm_src_11', u'adm_src_13', u'adm_src_14',
       u'adm_src_22', u'adm_src_25', u'adm_1', u'adm_2', u'adm_3', u'adm_4',
       u'adm_7', u'diss_home', u'num_lab_procedures', u'num_procedures',
       u'num_medications', u'number_outpatient', u'number_emergency',
       u'number_inpatient', u'number_diagnoses', u'insulin', u'metformin',
       u'pioglitazone', u'glimepiride', u'glipizide', u'repaglinide',
       u'nateglinide', u'readmitt

../src/MLpipeline.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_all["readmitted"][df_all["readmitted"].values > 0] = 1


In [ ]:
res = MLpipeline.run(df_all, colsNonDiseases, diseases, typeDataFeatures, 
               typeDataExperiment, typeEncounter, typeHypothesis, typeDiagnosis, ts_thr, tr_thrs, 
               fs_methods, sm_method, sm_types, 
               cls_methods, lms, cv_folds, cv_thr, True, False)

(2, 6)

DataSet:
**********
**********
SIZE: 1.0
DISEASE: subset
(67182, 52)
ALL TRAIN: (47027, 51)
TRAIN: [0's: 27849 1's: 19178 ]
ALL TEST: (20155, 51)
TEST: [0's: 11936 1's: 8219 ]

Num experiment: 0 / 1
****************
FS: none
SM: none
CLS: rf
METRIC: recall
Fitting 5 folds for each of 48 candidates, totalling 240 fits


/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/ilmira/.conda/envs/readmision/lib/python2.7/si

In [7]:
if 'dfAux' not in locals():
    print "Not dfAux"
    dfAux = pd.DataFrame()

if 'df' in locals():
    print "Exist df"
    dfAux = df.copy()
    
df = pd.DataFrame(np.array(res).reshape(len(res),27), columns=
                          ["exp", "typeDisease","typeEncounter","typeHypothesis","typeDataFeatures","typeDiagnosis",
                           "size_tr","fs","sm","cls","metric","params",
                           "tr_f1","tr_prec","tr_rec",
                           "cv_f1_mean","cv_f1_std","cv_prec_mean","cv_prec_std","cv_rec_mean","cv_rec_std",
                           "test_f1","test_prec","test_rec","test_auc",
                           "time","pipeline"])

df[["size_tr","sm","metric","cls","params", 'tr_f1', 'tr_prec', 'tr_rec', 
     'cv_f1_mean', 'cv_prec_mean', 'cv_rec_mean', 'test_f1', 'test_prec', 'test_rec', 'test_auc']]

Not dfAux


,size_tr,sm,metric,cls,params,tr_f1,tr_prec,tr_rec,cv_f1_mean,cv_prec_mean,cv_rec_mean,test_f1,test_prec,test_rec,test_auc
0,0.20,none,recall,rf,"[entropy, 8, 350]",0.68,0.68,0.68,0.60,0.60,0.60,0.61,0.61,0.61,0.60
1,0.20,none,f1_weighted,rf,"[gini, 12, 250]",0.84,0.84,0.84,0.61,0.60,0.61,0.61,0.61,0.61,0.60
2,0.20,none,recall,gbt,"[0.1, deviance, 0.1, 400, 12]",0.69,0.69,0.70,0.59,0.58,0.59,0.59,0.60,0.59,0.57
3,0.20,none,f1_weighted,gbt,"[0.1, deviance, 0.01, 400, 12]",0.61,0.66,0.65,0.59,0.61,0.62,0.59,0.63,0.62,0.57


In [8]:
df = pd.concat((dfAux, df))
df[["size_tr","sm","metric","cls","params",'tr_f1', 'tr_prec', 'tr_rec', 
     'cv_f1_mean', 'cv_prec_mean', 'cv_rec_mean', 'test_f1', 'test_prec', 
        'test_rec', 'test_auc']]

,size_tr,sm,metric,cls,params,tr_f1,tr_prec,tr_rec,cv_f1_mean,cv_prec_mean,cv_rec_mean,test_f1,test_prec,test_rec,test_auc
0,0.20,none,recall,rf,"[entropy, 8, 350]",0.68,0.68,0.68,0.60,0.60,0.60,0.61,0.61,0.61,0.60
1,0.20,none,f1_weighted,rf,"[gini, 12, 250]",0.84,0.84,0.84,0.61,0.60,0.61,0.61,0.61,0.61,0.60
2,0.20,none,recall,gbt,"[0.01, deviance, 0.1, 300, 6]",0.55,0.56,0.55,0.49,0.53,0.51,0.54,0.53,0.54,0.53
3,0.20,none,f1_weighted,gbt,"[0.5, deviance, 0.05, 300, 10]",0.67,0.70,0.69,0.60,0.61,0.62,0.60,0.63,0.62,0.58
